In [2]:
# split test into test_split and test_split_hidden
import pandas as pd
import random

train = pd.read_csv('data/train.csv')

# # shuffle test data
# train = test.sample(frac=1.0, random_state=22)

train_split_size = int(0.6 * len(train))

train_split = train[:train_split_size]
train_split_hidden = train[train_split_size:]

print(len(train_split), len(train_split_hidden))

# save files
train_split.to_csv('data/train_split.csv', index=False)
train_split_hidden.to_csv('data/test_split.csv', index=False)

534 357


In [ ]:
# write evaluation script

In [23]:
def evaluate(test_annotation_file, user_submission_file, phase_codename, **kwargs):
#     print(kwargs["submission_metadata"])
    
    test_file = "data/answers.csv"
    
    answers = pd.read_csv(test_file)
    user = pd.read_csv(user_submission_file)
    
    submission_result = ""
    
    result = {}
    result['result'] = []
    result["submission_result"] = submission_result
    
    if len(user) != len(answers):
        submission_result = "Number of rows in the training data ("+str(len(answers))+") and the submission file ("+str(len(user))+") don't match."
        result["submission_result"] = submission_result
        return result
    
    temp = {}
    temp[phase_codename] = {}
    matches = 0
    
    for i in range(len(user)):
        if user.iloc[i]['label'] == answers.iloc[i]['label']:
            matches = matches+1
            
    accuracy = round((matches/len(user))*100, 3)
    print("Accuracy:",accuracy)
    
    temp[phase_codename]['accuracy'] = accuracy
    result['result'].append(temp)
    submission_result = "Evaluated accuracy for "+str(phase_codename)+". Accuracy="+str(accuracy)
    result["submission_result"] = submission_result
    
    return result
    

In [24]:
result = evaluate('', 'data/submission.csv', 'phase1')

Accuracy: 10.071


In [25]:
print(result)

{'result': [{'phase1': {'accuracy': 10.071}}], 'submission_result': 'Evaluated accuracy for phase1. Accuracy=10.071'}


### Model for titanic

In [8]:
import pandas as pd
import numpy as np
import re

#Training (891 Entries) & Testing (417 Entries) data
train_data = pd.read_csv('data/train_split.csv')
test_data = pd.read_csv('data/test_split.csv')
all_data = [train_data, test_data]
passenger_id = test_data['PassengerId']

#Feature 3: Family Size
for data in all_data:
    data['family_size'] = data['SibSp'] + data['Parch'] + 1

#Feature 3.1: Is alone?
for data in all_data:
    data['is_alone'] = 0
    data.loc[data['family_size'] == 1, 'is_alone'] = 1
print (train_data[['is_alone', 'Survived']].groupby(['is_alone'], as_index=False).mean())

#Feature 4: Embarked
for data in all_data:
    data['Embarked'] = data['Embarked'].fillna('S')
print( train_data[["Embarked","Survived"]].groupby(["Embarked"], as_index = False).mean() )

#Feature 5: Fare
for data in all_data:
    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
train_data['category_fare'] = pd.qcut(train_data['Fare'], 4)
print( train_data[["category_fare","Survived"]].groupby(["category_fare"], as_index = False).mean() )

#Feature 6: Age
for data in all_data:
    age_avg  = data['Age'].mean()
    age_std  = data['Age'].std()
    age_null = data['Age'].isnull().sum()

    random_list = np.random.randint(age_avg - age_std, age_avg + age_std , size = age_null)
    data['Age'][np.isnan(data['Age'])] = random_list
    data['Age'] = data['Age'].astype(int)

train_data['category_age'] = pd.cut(train_data['Age'], 5)
print( train_data[["category_age","Survived"]].groupby(["category_age"], as_index = False).mean() )

#Feature 7: Name
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\. ', name)
    if title_search:
        return title_search.group(1)
    return ""

for data in all_data:
    data['title'] = data['Name'].apply(get_title)

for data in all_data:
    data['title'] = data['title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'],'Rare')
    data['title'] = data['title'].replace('Mlle','Miss')
    data['title'] = data['title'].replace('Ms','Miss')
    data['title'] = data['title'].replace('Mme','Mrs')
    
print(pd.crosstab(train_data['title'], train_data['Sex']))
print("----------------------")
print(train_data[['title','Survived']].groupby(['title'], as_index = False).mean())

#Map Data
for data in all_data:

    #Mapping Sex
    sex_map = { 'female':0 , 'male':1 }
    data['Sex'] = data['Sex'].map(sex_map).astype(int)

    #Mapping Title
    title_map = {'Mr':1, 'Miss':2, 'Mrs':3, 'Master':4, 'Rare':5}
    data['title'] = data['title'].map(title_map)
    data['title'] = data['title'].fillna(0)

    #Mapping Embarked
    embark_map = {'S':0, 'C':1, 'Q':2}
    data['Embarked'] = data['Embarked'].map(embark_map).astype(int)

    #Mapping Fare
    data.loc[ data['Fare'] <= 7.91, 'Fare']                            = 0
    data.loc[(data['Fare'] > 7.91) & (data['Fare'] <= 14.454), 'Fare'] = 1
    data.loc[(data['Fare'] > 14.454) & (data['Fare'] <= 31), 'Fare']   = 2
    data.loc[ data['Fare'] > 31, 'Fare']                               = 3
    data['Fare'] = data['Fare'].astype(int)

    #Mapping Age
    data.loc[ data['Age'] <= 16, 'Age']                       = 0
    data.loc[(data['Age'] > 16) & (data['Age'] <= 32), 'Age'] = 1
    data.loc[(data['Age'] > 32) & (data['Age'] <= 48), 'Age'] = 2
    data.loc[(data['Age'] > 48) & (data['Age'] <= 64), 'Age'] = 3
    data.loc[ data['Age'] > 64, 'Age']                        = 4

#Feature Selection
#Create list of columns to drop
drop_elements = ["Name", "Ticket", "Cabin", "SibSp", "Parch", "family_size"]

#Drop columns from both data sets
train_data = train_data.drop(drop_elements, axis = 1)
train_data = train_data.drop(['PassengerId','category_fare', 'category_age'], axis = 1)
test_data = test_data.drop(drop_elements, axis = 1)

#Print ready to use data
print(train_data.head(10))

X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()

#Running our classifier
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
accuracy = round(decision_tree.score(X_train, Y_train) * 100, 2)
print("Model Accuracy: ",accuracy)

   is_alone  Survived
0         0  0.479452
1         1  0.323810
  Embarked  Survived
0        C   0.51000
1        Q   0.48000
2        S   0.34375
       category_fare  Survived
0    (-0.001, 7.925]  0.248227
1    (7.925, 14.454]  0.307087
2   (14.454, 29.978]  0.462121
3  (29.978, 512.329]  0.537313
     category_age  Survived
0  (-0.071, 14.2]  0.566038
1    (14.2, 28.4]  0.339207
2    (28.4, 42.6]  0.423077
3    (42.6, 56.8]  0.326923
4    (56.8, 71.0]  0.300000


C:\Users\nabhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Sex     female  male
title               
Master       0    26
Miss       121     0
Mr           0   302
Mrs         77     0
Rare         0     8
----------------------
    title  Survived
0  Master  0.500000
1    Miss  0.710744
2      Mr  0.145695
3     Mrs  0.818182
4    Rare  0.125000
   Survived  Pclass  Sex  Age  Fare  Embarked  is_alone  title
0         0       3    1    1     0         0         0      1
1         1       1    0    2     3         1         0      3
2         1       3    0    1     1         0         1      2
3         1       1    0    2     3         0         0      3
4         0       3    1    2     1         0         1      1
5         0       3    1    1     1         2         1      1
6         0       1    1    3     3         0         1      1
7         0       3    1    0     2         0         0      4
8         1       3    0    1     1         0         0      3
9         1       2    0    0     2         1         0      3


ValueError: Number of features of the model must match the input. Model n_features is 7 and input n_features is 8 